# Notebook exploratório

- Artigo: Train one get one free
- Link: https://arxiv.org/abs/1903.12431

# Implementando rede

In [2]:
import torch
import transformers
import pytorch_lightning as pl
import torchtext as tt

In [ ]:
g_embedding = tt.vocab.GloVe()

In [20]:
g_embedding.get_vecs_by_tokens(['hello', 'what', 'is', 'up'], lower_case_backup=True)

tensor([[ 0.2523,  0.1018, -0.6748,  ...,  0.1787, -0.5192,  0.3359],
        [-0.0385,  0.5425, -0.2184,  ...,  0.1180,  0.2459,  0.2287],
        [-0.0850,  0.5020,  0.0024,  ..., -0.2151, -0.2630, -0.0060],
        [-0.0965,  0.1239, -0.2860,  ...,  0.0531,  0.0720, -0.3054]])